## Final project report: Finding a 'twin' neighborhood in Great Chicago area

### PART 1: Introduction <br>
In this final assignment,I will explore two areas: <br>
1)  Champaign, Urbana, and Savoy area in Illinois <br>
2) Great Chicago area in Illinois.<br>

The major aim is to see which area in Chicago is closest to my current living place (I am a resident in Champaign-Urbana-Savoy area). I am moving to Chicago in late July so I am very interested to know the 'twin' neighborhood there.<br>

I plan to solve this problem in these three steps:<br>

1) Obtrain the zipcode of all areas/neighborhoods in Chicago and Champaign-Urbana-Savoy.<br>
2) Obtain the coordinate that correspdone to these areas/zipcodes<br>
3) Using the coordinate information to get the venue located in these areas<br>
3) Perform a unsupervised machine learning method (for example, k-mean) to group those neighorhoods<br>
4) See if Champaign-Urbana-Savoy areas are grouped with some Chicago neighborhoods.<br>
5) Explore those corresponding'twin neighorhoods', including checking their locations on the map, check the venue distribution information, etc.<br>




### PART 2:  Data Introduction

I will be obtaining data from 4 resources: <br>
1) A publically available dataset provided by Opendatasoft, containing the coordinate informaiton of all zipcodes. I only obtained the Illinois data from this website: <br>
https://public.opendatasoft.com/explore/dataset/us-zip-code-latitude-and-longitude/table/?refine.state=IL <br>
This data is saved as 'IL_zip-code-latitude-and-longitude.csv' <br>

2) A website containing all Chicago area's zip code and the corresponding neighborhood name.<br>
https://www.seechicagorealestate.com/chicago-zip-codes-by-neighborhood.php#:~:text=Chicago%20real%20estate%20for%20sale,and%2060642%20for%20River%20North.'  <br>

3) A home-made excel file containing 5 local zip codes (my current place) and the corresponding neighborhood (city) names, saved as: <br>
'Champaign_Urbana_Savoy_ZIP_coordinates.xlsx' <br>

4) After the data cleaning and merging of the first 3 dataset, I will get a final dataset containing the cooridnates of each neighborhood in Great Chicago area as well as Champaign-Urbana-Savoy area. Then, venue data from Foursquare will be pulled (will be implemented in Week 5). The coordinate information will be used to get the information, then followed by unsupervised clustering.

### PART 3:  Methodology

####  3.1  Data obtaining, cleaning, and wrangling

In [1]:
import pandas as pd
import numpy as np
from pandas import Series, DataFrame

Dataset 1 processing:

In [2]:
#read the csv data.
IL_zipcode=pd.read_csv("C:/Users/qsun9/OneDrive/Documents/Python_learning/Python_coursera_DS_course_IBM/Course_4_Capstone/IL_zip-code-latitude-and-longitude.csv") 
IL_zipcode.head() 

,Zip;City;State;Latitude;Longitude;Timezone;Daylight savings time flag;geopoint
60651;Chicago;IL;41.901485;-87.74055;-6;1;41.901485,-87.74055
62090;Venice;IL;38.67065;-90.17052;-6;1;38.67065,-90.17052
62420;Casey;IL;39.299447;-87.99863;-6;1;39.299447,-87.99863
62206;East Saint Louis;IL;38.564451;-90.1628;-6;1;38.564451,-90.16280
62611;Arenzville;IL;39.883346;-90.39332;-6;1;39.883346,-90.39332


In [3]:
IL_zipcode.shape

(1627, 1)

The data needs cleaning. I will only keep the zip, city, state, latitude, and longitude.

In [4]:
IL_zipcode=IL_zipcode.reset_index()
Zipcode=[]
City=[]
State=[]
Latitude=[]
Longitude=[]

for i in range(0, len(IL_zipcode)):
    temp_split=IL_zipcode.iloc[i,0].split(";")
    Zipcode.append(temp_split[0])
    City.append(temp_split[1])
    State.append(temp_split[2])
    Latitude.append(temp_split[3])
    Longitude.append(temp_split[4])

IL_zipcode['Zip']=Zipcode
IL_zipcode['City']=City
IL_zipcode['State']=State
IL_zipcode['Latitude']=Latitude
IL_zipcode['Longitude']=Longitude

#make sure latitude and longitude are numeric.
IL_zipcode[['Latitude','Longitude']]=IL_zipcode[['Latitude','Longitude']].apply(pd.to_numeric, errors='coerce')

#drop unnecessary columns
IL_zipcode=IL_zipcode.drop(['index','Zip;City;State;Latitude;Longitude;Timezone;Daylight savings time flag;geopoint'], axis=1)

IL_zipcode.head()

,Zip,City,State,Latitude,Longitude
0,60651,Chicago,IL,41.901485,-87.74055
1,62090,Venice,IL,38.670650,-90.17052
2,62420,Casey,IL,39.299447,-87.99863
3,62206,East Saint Louis,IL,38.564451,-90.16280
4,62611,Arenzville,IL,39.883346,-90.39332


In [5]:
IL_zipcode.shape

(1627, 5)

In [6]:
len(IL_zipcode['Zip'].unique()) #all Zipcodes are unique.

1627

Dataset1 is clean. <br>


I now start the dataset 2: Chicago zipcode and neighborhood names

In [7]:
#dataset2: Chicago zipcode and area names
chicago_zipcode_area=pd.read_html('https://www.seechicagorealestate.com/chicago-zip-codes-by-neighborhood.php#:~:text=Chicago%20real%20estate%20for%20sale,and%2060642%20for%20River%20North.'
)[0]
chicago_zipcode_area.columns=['Area','Zip']
chicago_zipcode_area.shape

(199, 2)

Remove the 'non-zipcode' rows.

In [8]:
#Filter by checking if the first element in the string is numeric.
chicago_zipcode_area=chicago_zipcode_area.loc[chicago_zipcode_area["Zip"].str[0].astype(str).str.isnumeric(), ].reset_index(drop=True)

In [9]:
chicago_zipcode_area.head()

,Area,Zip
0,Cathedral District,60611
1,Central Station,60605
2,Dearborn Park,60605
3,Gold Coast,"60610, 60611"
4,Loop,"60601, 60602, 60603, 60604, 60605, 60606, 6060..."


Split multiple zipcodes that describe the same neighborhoods.

In [10]:
chicago_zipcode_area=pd.concat([Series(row['Area'], row['Zip'].split(',')) 
                                for _, row in chicago_zipcode_area.iterrows()]).reset_index()
chicago_zipcode_area.columns=['Zip','Area']

In [11]:
#fix utf encoding.
chicago_zipcode_area.Zip = chicago_zipcode_area.Zip.str.strip()

chicago_zipcode_area.Zip.str.encode('utf-8').head()
#now it looks good.

0    b'60611'
1    b'60605'
2    b'60605'
3    b'60610'
4    b'60611'
Name: Zip, dtype: object

We need the data to display unique zip codes instead of duplicated zip codes in different rows.<br>
However, there are duplicated zips in the data. I will group those neighborhoods' names.

In [12]:
chicago_zipcode_area=chicago_zipcode_area.groupby('Zip')['Area'].apply(','.join).reset_index()
chicago_zipcode_area.head()

,Zip,Area
0,60004,"Arlington Heights,Prospect Heights"
1,60005,Mount Prospect
2,60007,"Elk Grove Village,Schaumburg"
3,60009,Elk Grove Village
4,60010,"Barrington,Barrington Hills,Deer Park,Hoffman ..."


In [13]:
chicago_zipcode_area.shape

(179, 2)

In [14]:
len(chicago_zipcode_area['Zip'].unique())

179

now, the final dataset2 has 179 rows containing 179 unique Zipcodes.

Start processing dataset 3: the data that saves Champaign-Urbana-Savoy zipcode and area names.

In [15]:
#Dataset3: Champaign-Urbana-Savoy zip code area data

CUS_zipcode_area=pd.read_excel("C:/Users/qsun9/OneDrive/Documents/Python_learning/Python_coursera_DS_course_IBM/Course_4_Capstone/Champaign_Urbana_Savoy_ZIP_coordinates.xlsx") 
CUS_zipcode_area

,Zip,Area
0,61820,Champaign
1,61821,Champaign
2,61822,Champaign
3,61801,Urbana
4,61802,Urbana
5,61874,Savoy


Combine Dataset2 chicago_zipcode_area and Dataset3 CUS_zipcode_area

In [16]:
Chi_CUS_zip_area=pd.concat([chicago_zipcode_area,CUS_zipcode_area]).reset_index(drop=True)
Chi_CUS_zip_area.shape 

(185, 2)

Now, the dataset contains 185 zipcodes and the corresponding area names

Finish of the data cleaning. <br>

Now, Overlap Chi_CUS_zip_area with the Dataset1. 

In [17]:
#the last 6 zip are not 'str' 
(Chi_CUS_zip_area.applymap(type)==str).iloc[175:,]

,Zip,Area
175,True,True
176,True,True
177,True,True
178,True,True
179,False,True
180,False,True
181,False,True
182,False,True
183,False,True
184,False,True


In [18]:
#Change it back to 'str'
Chi_CUS_zip_area.Zip = Chi_CUS_zip_area.Zip.astype(str).str.strip()

Now we are good to merge the two datasets together.

In [19]:
Final_area_df=Chi_CUS_zip_area.join(IL_zipcode.set_index('Zip'), on='Zip')

In [21]:
#remove the NA value rows
Final_area_df=Final_area_df.dropna()
Final_area_df.shape

(176, 6)

In [22]:
Final_area_df.head()  
#176 final rows

,Zip,Area,City,State,Latitude,Longitude
0,60004,"Arlington Heights,Prospect Heights",Arlington Heights,IL,42.108428,-87.97723
1,60005,Mount Prospect,Arlington Heights,IL,42.069327,-87.98464
2,60007,"Elk Grove Village,Schaumburg",Elk Grove Village,IL,42.005978,-87.99847
3,60009,Elk Grove Village,Elk Grove Village,IL,41.811929,-87.68732
4,60010,"Barrington,Barrington Hills,Deer Park,Hoffman ...",Barrington,IL,42.160791,-88.15231


Now, I will use the information in 'Final_area_df' to extract the venues located in each area/neighborhood

In [24]:
import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [25]:
#first, drop the 'city' column.
Final_area_df=Final_area_df.drop(['City','State'], axis=1)

Let's visualize those areas on the map

In [26]:
#Obtain the geographical coordinates of a place inbetween Champaign and Great Chicago: Bourbonnais.

address = 'Bourbonnais, Illinois'
geolocator = Nominatim(user_agent="chicago_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Bourbonnais are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Bourbonnais are 41.1639309, -87.8839693.


In [51]:
# create map to include the Champaign-Urbana-Savoy area and the Great Chicago area.
map_chicago_CU = folium.Map(location=[latitude, longitude], zoom_start=8)

# add markers to map
for lat, lng, postal_code, area in zip(Final_area_df['Latitude'], Final_area_df['Longitude'],Final_area_df['Zip'], Final_area_df['Area']):
    label = '{},{}'.format(postal_code, area)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_chicago_CU)  

    #visualize the map 
map_chicago_CU

The map above shows tha the areas in Champaign-Urbana-Savoy area located in the sounth side of Chicago area.

Now, we can use the 

In [28]:
#Use foursquare API to explore these areas.
CLIENT_ID = #hidden for security purposes#
CLIENT_SECRET =  #hidden for security purposes#
VERSION = '20180605'

In [29]:
#explore the venues in the neighborhood
#create a function to extract the top 100 venues within 1000 meters of these areas.
radius=1000
LIMIT = 100

def getNearbyVenues(zipcodes, names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for zipcode,name, lat, lng in zip(zipcodes, names, latitudes, longitudes):
        print(zipcode+','+ name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            zipcode,
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Zipcode',
                             'Area',
                             'Area Latitude',
                             'Area Longitude',
                             'Venue',
                             'Venue Latitude',
                             'Venue Longitude',
                             'Venue Category']
    
    return(nearby_venues)

In [32]:
# use the function created above to pull the venue information.

my_venues = getNearbyVenues(zipcodes=Final_area_df['Zip'],
                            names=Final_area_df['Area'],
                            latitudes=Final_area_df['Latitude'],
                            longitudes=Final_area_df['Longitude']
                                  )

60004,Arlington Heights,Prospect Heights
60005,Mount Prospect
60007,Elk Grove Village,Schaumburg
60009,Elk Grove Village
60010,Barrington,Barrington Hills,Deer Park,Hoffman Estates,Iverness,South Barrington
60012,Crystal Lake
60014,Crystal Lake
60015,Bannockburn,Deerfield
60016,Glenview,Des Plaines,Park Ridge
60017,Des Plaines
60018,Des Plaines,Rosemont
60019,Des Plaines
60022,Glencoe
60025,Glenview
60026,Glenview,Northbrook
60029,Glenview,Golf
60030,Grayslake
60035,Highland Park,Lake Forest
60039,Crystal Lake
60040,Highland Park,Highwood
60043,Kenilworth,Winnetka
60044,Lake Bluff
60045,Lake Forest
60047,Lake Zurich,Long Grove
60049,Long Grove
60053,Morton Grove
60056,Mount Prospect
60060,Long Grove
60062,Glenview,Northbrook,Northbrook,Prospect Heights
60065,Northbrook,Northbrook
60067,Hoffman Estates,Iverness
60068,Park Ridge
60070,Prospect Heights
60076,Skokie
60077,Skokie
60082,Glenview
60089,Buffalo Grove
60090,Prospect Heights
60091,Wilmette
60093,Glencoe,Northfield,Winnetka
60102

In [33]:
print(my_venues.shape)
my_venues.head()

(6765, 8)


,Zipcode,Area,Area Latitude,Area Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,60004,"Arlington Heights,Prospect Heights",42.108428,-87.97723,LA Fitness,42.107957,-87.978448,Gym
1,60004,"Arlington Heights,Prospect Heights",42.108428,-87.97723,Walgreens,42.107892,-87.979809,Pharmacy
2,60004,"Arlington Heights,Prospect Heights",42.108428,-87.97723,Olive Garden,42.109389,-87.974479,Italian Restaurant
3,60004,"Arlington Heights,Prospect Heights",42.108428,-87.97723,AT&T,42.109136,-87.976522,Mobile Phone Shop
4,60004,"Arlington Heights,Prospect Heights",42.108428,-87.97723,Starbucks,42.108327,-87.981104,Coffee Shop


In [34]:
my_venues.loc[my_venues['Area']=='Champaign',].shape

(143, 8)

In [35]:
my_venues.loc[my_venues['Area']=='Urbana',].shape

(80, 8)

In [36]:
my_venues.loc[my_venues['Area']=='Savoy',].shape

(27, 8)

In [37]:
#check the count of venue categories
#notice: some zipcodes have same 'Area' names, hence we have to group by zip here. 
my_venues.groupby('Zipcode').count() 

,Area,Area Latitude,Area Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Zipcode,,,,,,,
60004,81,81,81,81,81,81,81
60005,11,11,11,11,11,11,11
60007,37,37,37,37,37,37,37
60009,37,37,37,37,37,37,37
60010,21,21,21,21,21,21,21
60012,4,4,4,4,4,4,4
60014,48,48,48,48,48,48,48
60015,7,7,7,7,7,7,7
60016,37,37,37,37,37,37,37


In [39]:
print('There are {} uniques categories.'.format(len(my_venues['Venue Category'].unique())))

There are 392 uniques categories.


In [40]:
#analysis of the Neighborhoods by using one-hot encoding

# one hot encoding
my_onehot = pd.get_dummies(my_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
my_onehot['Zipcode'] = my_venues['Zipcode'] 
my_onehot['Area'] = my_venues['Area'] 


# move last two columns to the first 
fixed_columns = [my_onehot.columns[-2]] + [my_onehot.columns[-1]]+  list(my_onehot.columns[:-2])
my_onehot = my_onehot[fixed_columns]
my_onehot.shape


(6765, 394)

In [42]:
#group rows by zipcode and by taking the mean of the frequency of occurrence of each category
my_onehot.drop(['Area'], axis=1)
my_grouped = my_onehot.groupby('Zipcode').mean().reset_index()
my_grouped.head()

,Zipcode,ATM,Accessories Store,Adult Boutique,Advertising Agency,African Restaurant,Airport,Airport Terminal,American Restaurant,Amphitheater,...,Waterfront,Weight Loss Center,Whisky Bar,Wine Bar,Wine Shop,Winery,Wings Joint,Women's Store,Yoga Studio,Zoo Exhibit
0,60004,0.0,0.012346,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.012346,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0
1,60005,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0
2,60007,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.027027,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.027027,0.000000,0.0,0.0
3,60009,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.027027,0.0,0.0
4,60010,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0


In [43]:
my_grouped.shape  #176 Zipcode, and 304 unique venues.

(176, 393)

We can print each Zipcode along with the top 5 most common venues.

In [44]:
# print each neighborhood along with the top 5 most common venues
num_top_venues = 5

for current_zip in my_grouped['Zipcode']:
    print("----"+current_zip+"----")
    temp = my_grouped[my_grouped['Zipcode'] == current_zip].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----60004----
                    venue  freq
0          Clothing Store  0.04
1  Thrift / Vintage Store  0.04
2                    Bank  0.04
3       Mobile Phone Shop  0.04
4    Fast Food Restaurant  0.04


----60005----
                venue  freq
0         Pizza Place  0.18
1         Bridal Shop  0.09
2   Convenience Store  0.09
3  Chinese Restaurant  0.09
4      Sandwich Place  0.09


----60007----
                venue  freq
0      Sandwich Place  0.14
1  Mexican Restaurant  0.11
2      Shipping Store  0.05
3   Mobile Phone Shop  0.05
4              Bakery  0.03


----60009----
               venue  freq
0        Gas Station  0.08
1       Liquor Store  0.05
2     Farmers Market  0.03
3  Convenience Store  0.03
4           Pharmacy  0.03


----60010----
                        venue  freq
0  Financial or Legal Service  0.14
1                Liquor Store  0.10
2         Rental Car Location  0.05
3            Asian Restaurant  0.05
4                     Stadium  0.05


----60012----


                 venue  freq
0                 Bank  0.14
1  American Restaurant  0.11
2             Pharmacy  0.07
3          Video Store  0.04
4          Supermarket  0.04


----60122----
             venue  freq
0       Hobby Shop  0.12
1  Doctor's Office  0.12
2       Toll Plaza  0.12
3  Automotive Shop  0.12
4           Lawyer  0.12


----60123----
                  venue  freq
0                  Bank  0.21
1  Gym / Fitness Center  0.11
2              Pharmacy  0.11
3            Donut Shop  0.05
4           Video Store  0.05


----60126----
                venue  freq
0         Pizza Place  0.14
1                Park  0.07
2         Coffee Shop  0.07
3                 ATM  0.03
4  Chinese Restaurant  0.03


----60137----
                venue  freq
0               Diner  0.08
1  Italian Restaurant  0.08
2  Mexican Restaurant  0.08
3                 ATM  0.04
4       Burrito Place  0.04


----60138----
                  venue  freq
0  Gym / Fitness Center  0.17
1        Baseball Fi

                  venue  freq
0          Intersection  0.07
1  Fast Food Restaurant  0.07
2   Rental Car Location  0.07
3   Sporting Goods Shop  0.05
4    Chinese Restaurant  0.05


----60516----
                           venue  freq
0            Arts & Crafts Store  0.09
1  Paper / Office Supplies Store  0.06
2         Furniture / Home Store  0.06
3           Fast Food Restaurant  0.06
4                           Park  0.06


----60517----
                 venue  freq
0         Intersection  0.10
1  Sporting Goods Shop  0.10
2    Convenience Store  0.10
3                Diner  0.05
4       Breakfast Spot  0.05


----60521----
         venue  freq
0  Music Store  0.25
1         Farm  0.25
2     Hospital  0.25
3  Coffee Shop  0.25
4          ATM  0.00


----60522----
                  venue  freq
0  Gym / Fitness Center  0.17
1        Baseball Field  0.17
2           Coffee Shop  0.17
3            Water Park  0.17
4            Comic Shop  0.17


----60523----
                    venue 

                       venue  freq
0             Sandwich Place  0.06
1                Coffee Shop  0.05
2                       Park  0.04
3  Middle Eastern Restaurant  0.04
4                Pizza Place  0.04


----60630----
            venue  freq
0             Bar  0.10
1  Ice Cream Shop  0.06
2        Pharmacy  0.06
3            Park  0.05
4   Train Station  0.03


----60631----
                venue  freq
0  Italian Restaurant  0.07
1                 Bar  0.07
2      Breakfast Spot  0.04
3              Bakery  0.04
4                Bank  0.04


----60634----
               venue  freq
0        Pizza Place  0.10
1    Thai Restaurant  0.05
2  Convenience Store  0.05
3       Liquor Store  0.05
4      Deli / Bodega  0.05


----60636----
                  venue  freq
0  Fast Food Restaurant  0.25
1          Liquor Store  0.19
2        Sandwich Place  0.12
3           Pizza Place  0.06
4         Women's Store  0.06


----60637----
                  venue  freq
0                  Park  0

In [45]:
#write a function to sort the venues in descending order.
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Let's create the new dataframe calle 'neighborhoods_venues_sorted' and display the top 10 venues for each zipcode.

In [46]:
#create the new dataframe and display the top 10 venues for each zipcode.
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Zipcode']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Zipcode'] = my_grouped['Zipcode']

#now, loop each of the 39 areas/neighborhood_names to fill in the top 10 venue cateogires.
for ind in np.arange(my_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(my_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Zipcode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,60004,Thrift / Vintage Store,Furniture / Home Store,Bank,Mobile Phone Shop,Fast Food Restaurant,Clothing Store,Gas Station,Shipping Store,Coffee Shop,Grocery Store
1,60005,Pizza Place,Bridal Shop,Gym,Chinese Restaurant,Hot Dog Joint,Cosmetics Shop,Food Court,Convenience Store,Home Service,Sandwich Place
2,60007,Sandwich Place,Mexican Restaurant,Mobile Phone Shop,Shipping Store,Concert Hall,Racetrack,Bank,Supermarket,Sports Bar,Salon / Barbershop
3,60009,Gas Station,Liquor Store,Metro Station,Rental Car Location,Coffee Shop,Bakery,Clothing Store,Thrift / Vintage Store,Laundromat,Sandwich Place
4,60010,Financial or Legal Service,Liquor Store,Stadium,Restaurant,Bridal Shop,Mexican Restaurant,Business Service,Café,Office,Baseball Field


We can quickly check the Champaign-Urbana-Savoy area because they are in the end.

In [47]:
neighborhoods_venues_sorted.tail()

,Zipcode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
171,61802,Garden Center,Baseball Field,Pizza Place,Zoo Exhibit,Event Space,Exhibit,Eye Doctor,Fabric Shop,Falafel Restaurant,Farm
172,61820,Bar,Chinese Restaurant,Pizza Place,Mexican Restaurant,Park,Coffee Shop,Liquor Store,Indian Restaurant,Convenience Store,Supermarket
173,61821,Convenience Store,Fast Food Restaurant,Discount Store,Pizza Place,BBQ Joint,Pharmacy,Dry Cleaner,Donut Shop,Bar,Breakfast Spot
174,61822,Home Service,Theater,Park,College Gym,Gym,Planetarium,Soccer Field,Discount Store,Food Truck,Exhibit
175,61874,Hotel,Massage Studio,Playground,Park,Mexican Restaurant,Sporting Goods Shop,Fast Food Restaurant,Bowling Alley,Supermarket,Supplement Shop


In [49]:
#add the area name and coordinates using information from Final_area_df.
neighborhoods_venues_sorted=neighborhoods_venues_sorted.join(Final_area_df.set_index('Zip'), on='Zipcode')
neighborhoods_venues_sorted=neighborhoods_venues_sorted[list(neighborhoods_venues_sorted.columns[-3:])  + list(neighborhoods_venues_sorted.columns[:-3])]

Now, data is good go to be used in the clustering analysis.

####  3.2  Clustering of all areas using unsupervised machine learning (k-Mean)

Now, I will perform  k-means using the 'my_grouped' dataframe. Here I choose 6.

In [64]:
kclusters =6

my_grouped_clustering = my_grouped.drop('Zipcode', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(my_grouped_clustering)


### PART 4: Results

First, let's check the number of neighborhoods in each cluster 

In [65]:
unique, counts = np.unique(kmeans.labels_, return_counts=True)
dict(zip(unique, counts))

{0: 25, 1: 3, 2: 8, 3: 71, 4: 6, 5: 63}

From this results above, we can see that three clusters include relatively large number of areas and another three cluster contain only a few. We will explore those clustering results to check what those 'rare' clusters are, and we will check where we can find Champaign-Urbana-Savoy in those clusters.

In [59]:
#  add clustering labels on the dataframe.
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Let's visualize the clustering results on the map.

In [67]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=8)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for zipcode, lat, lon, poi, cluster in zip(neighborhoods_venues_sorted['Zipcode'], neighborhoods_venues_sorted['Latitude'], neighborhoods_venues_sorted['Longitude'], neighborhoods_venues_sorted['Area'], neighborhoods_venues_sorted['Cluster Labels']):
    label = folium.Popup(str(zipcode)+',' +str(poi)  +' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Let's check my areas' 6 zipcodes to see what their clusters are.

In [63]:
neighborhoods_venues_sorted.tail(n=6)

,Cluster Labels,Area,Latitude,Longitude,Zipcode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
170,3,Urbana,40.109647,-88.21246,61801,Bar,Sandwich Place,Food Truck,American Restaurant,Pizza Place,Vegetarian / Vegan Restaurant,Ice Cream Shop,Bank,BBQ Joint,Coffee Shop
171,1,Urbana,40.121648,-88.17649,61802,Garden Center,Baseball Field,Pizza Place,Zoo Exhibit,Event Space,Exhibit,Eye Doctor,Fabric Shop,Falafel Restaurant,Farm
172,3,Champaign,40.114931,-88.24322,61820,Bar,Chinese Restaurant,Pizza Place,Mexican Restaurant,Park,Coffee Shop,Liquor Store,Indian Restaurant,Convenience Store,Supermarket
173,5,Champaign,40.108631,-88.27330,61821,Convenience Store,Fast Food Restaurant,Discount Store,Pizza Place,BBQ Joint,Pharmacy,Dry Cleaner,Donut Shop,Bar,Breakfast Spot
174,0,Champaign,40.126854,-88.29315,61822,Home Service,Theater,Park,College Gym,Gym,Planetarium,Soccer Field,Discount Store,Food Truck,Exhibit
175,5,Savoy,40.061694,-88.25023,61874,Hotel,Massage Studio,Playground,Park,Mexican Restaurant,Sporting Goods Shop,Fast Food Restaurant,Bowling Alley,Supermarket,Supplement Shop


It is very interesting to see that the 6 areas in Champaign-Urbana-Savoy are in three different clusters. <br>
Two zipcodes associated with Urbana are in cluster 3 and 1 respectively. 3 zipcodes associated with Champaign are clustered into three complete different clusters as well. Savoy is clustered into cluster 5.<br>
And, the previous analysis showed that the number of areas in each cluster is: 0: 25, 1: 3, 2: 8, 3: 71, 4: 6, 5: 63 <br>
Let's also print those details in the clustering results one by one, and discuss about them in the PART 4.

##### Cluster 0:

In [86]:
#cluster 0
neighborhoods_venues_sorted.loc[neighborhoods_venues_sorted['Cluster Labels'] == 0, ]

,Cluster Labels,Area,Latitude,Longitude,Zipcode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,0,Crystal Lake,42.265643,-88.316640,60012,Park,Fishing Spot,Home Service,Massage Studio,Zoo Exhibit,Fast Food Restaurant,Event Service,Event Space,Exhibit,Eye Doctor
13,0,Glenview,42.076720,-87.819220,60025,Salon / Barbershop,Park,Steakhouse,Eye Doctor,Bank,Gym / Fitness Center,Sandwich Place,Coffee Shop,Supermarket,Convenience Store
20,0,"Kenilworth,Winnetka",42.088128,-87.716000,60043,Trail,Park,Sandwich Place,Ice Cream Shop,Yoga Studio,Beach,Golf Course,Convenience Store,French Restaurant,Train Station
22,0,Lake Forest,42.238087,-87.860930,60045,Park,Intersection,Home Service,Theater,Fast Food Restaurant,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Eye Doctor
28,0,"Glenview,Northbrook,Northbrook,Prospect Heights",42.124576,-87.843030,60062,Baseball Field,Park,Food,Sporting Goods Shop,Mobile Phone Shop,Locksmith,Athletics & Sports,Gym / Fitness Center,Business Service,Skating Rink
35,0,Glenview,42.116377,-87.812064,60082,Golf Course,Hotel,Park,Spa,Pool,Skating Rink,Furniture / Home Store,Tennis Court,Optical Shop,Light Rail Station
36,0,Buffalo Grove,42.167638,-87.963930,60089,Park,Farmers Market,Fast Food Restaurant,Golf Course,Gym,Skate Park,Baseball Field,Gas Station,Disc Golf,Event Service
38,0,Wilmette,42.077178,-87.723730,60091,Park,Pizza Place,Spa,Coffee Shop,Asian Restaurant,Pet Store,Diner,Grocery Store,Butcher,Convenience Store
41,0,"Bartlett,Streamwood",41.977477,-88.172570,60103,Baseball Field,Video Store,Bank,Gym,Park,Convenience Store,Business Service,Coffee Shop,Zoo Exhibit,Exhibit
42,0,"Hoffman Estates,Streamwood",42.023977,-88.176570,60107,Baseball Field,Park,Gym,Gym / Fitness Center,Italian Restaurant,Locksmith,Liquor Store,Disc Golf,Tailor Shop,Sports Bar


##### Cluster 1:

In [85]:
#cluster 1
neighborhoods_venues_sorted.loc[neighborhoods_venues_sorted['Cluster Labels'] == 1, ]

,Cluster Labels,Area,Latitude,Longitude,Zipcode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
95,1,Aurora,41.936977,-88.420178,60507,Trail,Home Service,Park,Bar,Baseball Field,Garden Center,Fishing Spot,Falafel Restaurant,Event Service,Event Space
120,1,Aurora,41.935616,-88.432380,60568,Automotive Shop,Park,Bar,Baseball Field,Garden Center,Fast Food Restaurant,Event Space,Exhibit,Eye Doctor,Fabric Shop
171,1,Urbana,40.121648,-88.176490,61802,Garden Center,Baseball Field,Pizza Place,Zoo Exhibit,Event Space,Exhibit,Eye Doctor,Fabric Shop,Falafel Restaurant,Farm


##### Cluster 2:

In [87]:
#cluster 2
neighborhoods_venues_sorted.loc[neighborhoods_venues_sorted['Cluster Labels'] == 2, ]

,Cluster Labels,Area,Latitude,Longitude,Zipcode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
52,2,Glen Ellyn,41.839679,-88.088716,60138,Gym,Gym / Fitness Center,Baseball Field,Comic Shop,Water Park,Coffee Shop,Zoo Exhibit,Farmers Market,Event Service,Event Space
70,2,Wheaton,41.839679,-88.088716,60189,Gym,Gym / Fitness Center,Baseball Field,Comic Shop,Water Park,Coffee Shop,Zoo Exhibit,Farmers Market,Event Service,Event Space
101,2,"Hinsdale,Oak Brook,Oakbrook Terrace",41.839679,-88.088716,60522,Gym,Gym / Fitness Center,Baseball Field,Comic Shop,Water Park,Coffee Shop,Zoo Exhibit,Farmers Market,Event Service,Event Space
105,2,"Burr Ridge,Willowbrook",41.839679,-88.088716,60527,Gym,Gym / Fitness Center,Baseball Field,Comic Shop,Water Park,Coffee Shop,Zoo Exhibit,Farmers Market,Event Service,Event Space
118,2,Naperville,41.839679,-88.088716,60566,Gym,Gym / Fitness Center,Baseball Field,Comic Shop,Water Park,Coffee Shop,Zoo Exhibit,Farmers Market,Event Service,Event Space
119,2,Naperville,41.839679,-88.088716,60567,Gym,Gym / Fitness Center,Baseball Field,Comic Shop,Water Park,Coffee Shop,Zoo Exhibit,Farmers Market,Event Service,Event Space
121,2,Aurora,41.839679,-88.088716,60572,Gym,Gym / Fitness Center,Baseball Field,Comic Shop,Water Park,Coffee Shop,Zoo Exhibit,Farmers Market,Event Service,Event Space
122,2,Aurora,41.839679,-88.088716,60598,Gym,Gym / Fitness Center,Baseball Field,Comic Shop,Water Park,Coffee Shop,Zoo Exhibit,Farmers Market,Event Service,Event Space


##### Cluster 3:

In [88]:
#cluster 3
neighborhoods_venues_sorted.loc[neighborhoods_venues_sorted['Cluster Labels'] == 3,]

,Cluster Labels,Area,Latitude,Longitude,Zipcode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,3,Mount Prospect,42.069327,-87.984640,60005,Pizza Place,Bridal Shop,Gym,Chinese Restaurant,Hot Dog Joint,Cosmetics Shop,Food Court,Convenience Store,Home Service,Sandwich Place
10,3,"Des Plaines,Rosemont",42.008429,-87.892340,60018,Hotel,Sandwich Place,Coffee Shop,Stadium,Park,Café,Pizza Place,Business Service,Mobile Phone Shop,Rental Car Location
12,3,Glencoe,42.130976,-87.762520,60022,Coffee Shop,Gourmet Shop,Business Service,Bank,Chinese Restaurant,Golf Course,Locksmith,Trail,Train Station,Grocery Store
15,3,"Glenview,Golf",42.056529,-87.792860,60029,Pharmacy,Massage Studio,Thai Restaurant,Bar,Grocery Store,Train Station,Korean Restaurant,Toy / Game Store,Bakery,Bank
16,3,Grayslake,42.338955,-88.034330,60030,Convenience Store,Wine Bar,Coffee Shop,Ice Cream Shop,American Restaurant,Toy / Game Store,Gas Station,Pizza Place,Library,Beer Garden
17,3,"Highland Park,Lake Forest",42.181875,-87.809560,60035,Ice Cream Shop,Sandwich Place,Sporting Goods Shop,BBQ Joint,Pizza Place,Salon / Barbershop,Bagel Shop,Bakery,Bank,Thrift / Vintage Store
19,3,"Highland Park,Highwood",42.205724,-87.814210,60040,Italian Restaurant,American Restaurant,Bar,Pizza Place,Mexican Restaurant,Bakery,Asian Restaurant,Coffee Shop,Chinese Restaurant,Liquor Store
23,3,"Lake Zurich,Long Grove",42.199957,-88.058590,60047,Flower Shop,Pizza Place,Grocery Store,Park,Sandwich Place,Coffee Shop,Café,Fast Food Restaurant,Butcher,Filipino Restaurant
25,3,Morton Grove,42.041999,-87.788820,60053,Pizza Place,Trail,Coffee Shop,Furniture / Home Store,Sporting Goods Shop,Sushi Restaurant,Thrift / Vintage Store,Restaurant,Supermarket,Bar
26,3,Mount Prospect,42.065427,-87.936210,60056,Park,Italian Restaurant,Breakfast Spot,Sandwich Place,Ice Cream Shop,Toy / Game Store,Restaurant,Train Station,Greek Restaurant,Coffee Shop


##### Cluster 4:

In [89]:
#cluster 4
neighborhoods_venues_sorted.loc[neighborhoods_venues_sorted['Cluster Labels'] ==4, ]

,Cluster Labels,Area,Latitude,Longitude,Zipcode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,4,"Bannockburn,Deerfield",42.169325,-87.865560,60015,Park,Pool,Gym,Dog Run,Outdoors & Recreation,Flea Market,Fabric Shop,Empanada Restaurant,Ethiopian Restaurant,Flower Shop
21,4,Lake Bluff,42.286222,-87.863090,60044,Park,Golf Course,Trail,Zoo Exhibit,Farmers Market,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Eye Doctor
39,4,"Glencoe,Northfield,Winnetka",42.104127,-87.750160,60093,Park,Trail,Flower Shop,Golf Course,Baseball Field,Theater,Skating Rink,Tennis Court,Falafel Restaurant,Event Service
59,4,Lake in the Hills,42.185733,-88.348484,60156,Park,Golf Course,Baseball Field,Hardware Store,Zoo Exhibit,Empanada Restaurant,Event Service,Event Space,Exhibit,Eye Doctor
72,4,Schaumburg,42.012990,-88.096750,60193,Park,Breakfast Spot,Italian Restaurant,Field,Event Service,Event Space,Exhibit,Eye Doctor,Fabric Shop,Falafel Restaurant
74,4,"Hoffman Estates,Schaumburg",42.067809,-88.108280,60195,Park,Moving Target,Nature Preserve,Business Service,Floating Market,Farmers Market,Fondue Restaurant,Event Service,Event Space,Exhibit


##### Cluster 5:

In [90]:
#cluster 5
neighborhoods_venues_sorted.loc[neighborhoods_venues_sorted['Cluster Labels'] == 5, ]

,Cluster Labels,Area,Latitude,Longitude,Zipcode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,5,"Arlington Heights,Prospect Heights",42.108428,-87.977230,60004,Thrift / Vintage Store,Furniture / Home Store,Bank,Mobile Phone Shop,Fast Food Restaurant,Clothing Store,Gas Station,Shipping Store,Coffee Shop,Grocery Store
2,5,"Elk Grove Village,Schaumburg",42.005978,-87.998470,60007,Sandwich Place,Mexican Restaurant,Mobile Phone Shop,Shipping Store,Concert Hall,Racetrack,Bank,Supermarket,Sports Bar,Salon / Barbershop
3,5,Elk Grove Village,41.811929,-87.687320,60009,Gas Station,Liquor Store,Metro Station,Rental Car Location,Coffee Shop,Bakery,Clothing Store,Thrift / Vintage Store,Laundromat,Sandwich Place
4,5,"Barrington,Barrington Hills,Deer Park,Hoffman ...",42.160791,-88.152310,60010,Financial or Legal Service,Liquor Store,Stadium,Restaurant,Bridal Shop,Mexican Restaurant,Business Service,Café,Office,Baseball Field
6,5,Crystal Lake,42.226623,-88.330660,60014,Pizza Place,Mexican Restaurant,Thrift / Vintage Store,Automotive Shop,Grocery Store,Gas Station,Fried Chicken Joint,American Restaurant,Fast Food Restaurant,Hardware Store
8,5,"Glenview,Des Plaines,Park Ridge",42.047178,-87.890580,60016,Mexican Restaurant,Bakery,Sandwich Place,ATM,Gastropub,Discount Store,Grocery Store,Greek Restaurant,Donut Shop,Bar
9,5,Des Plaines,42.028779,-87.894366,60017,Pizza Place,Fast Food Restaurant,Liquor Store,Bar,Playground,Coffee Shop,Burger Joint,Bank,Salon / Barbershop,Grocery Store
11,5,Des Plaines,42.024278,-87.907066,60019,Pizza Place,Hobby Shop,Thai Restaurant,Fast Food Restaurant,Convenience Store,Food,Automotive Shop,Latin American Restaurant,Bakery,Bank
14,5,"Glenview,Northbrook",41.811929,-87.687320,60026,Gas Station,Liquor Store,Metro Station,Rental Car Location,Coffee Shop,Bakery,Clothing Store,Thrift / Vintage Store,Laundromat,Sandwich Place
18,5,Crystal Lake,42.324761,-88.452481,60039,Liquor Store,American Restaurant,Gas Station,Mexican Restaurant,Gym,Train Station,Tennis Court,Grocery Store,Home Service,Hot Dog Joint


### PART 5: Discussion 

In order to better disuss my findings, I will show two locations in two individual maps as below:

In [68]:
location_Chicago = geolocator.geocode('Chicago,Illinois')
location_Champaign = geolocator.geocode('Champaign, Illinois')
latitude_Chicago = location_Chicago.latitude
longitude_Chicago = location_Chicago.longitude
latitude_Champaign = location_Champaign.latitude
longitude_Champaign = location_Champaign.longitude

First, create Great Chicago map to visualize the clustering in this region.

In [91]:
# create map
map_clusters = folium.Map(location=[latitude_Chicago, longitude_Chicago], zoom_start=9)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for zipcode, lat, lon, poi, cluster in zip(neighborhoods_venues_sorted['Zipcode'], neighborhoods_venues_sorted['Latitude'], neighborhoods_venues_sorted['Longitude'], neighborhoods_venues_sorted['Area'], neighborhoods_venues_sorted['Cluster Labels']):
    label = folium.Popup(str(zipcode)+',' +str(poi)  +' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Then let's visualize Champaign-Urbana-Savoy area only.

In [92]:
# create map
map_clusters = folium.Map(location=[latitude_Champaign, longitude_Champaign], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for zipcode, lat, lon, poi, cluster in zip(neighborhoods_venues_sorted['Zipcode'], neighborhoods_venues_sorted['Latitude'], neighborhoods_venues_sorted['Longitude'], neighborhoods_venues_sorted['Area'], neighborhoods_venues_sorted['Cluster Labels']):
    label = folium.Popup(str(zipcode)+',' +str(poi)  +' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Cluster 0 has 25 zip codes/areas where Park is the top venue. You also see golf course and other sports-related venues and home services and restaurants, etc. These categories indicate that cluster 0 is like suburb areas, mostly including outdoor activity venues as well as some some venues indicating residential areas. From the Chicago map (red dots) we can see that these red dots are scattered away from the downtown area. We can see that one of the two Champaign zip code, 61822, falls into cluster 0. This region is exactly like the suburb with residential area. <br>

Cluster 1 has only three zip codes/areas, including one Urbana zip code: 61802. I noticed that 61802 has only three venues found, including Pizza Place, Baseball Field and Garden Center (one of each). Hence the very 'sparse' results will largely affect the analysis results. I think the reason why those two Aurora areas are clustered together with Urbana-61802 is because they both contain 'Garden center'. The future plan for this project is to design another metrics including not only the frequency of the venues within each location, but also take the total number of venues into account in some way. Because the density of venues will also be meaningful in the area clustering problem. <br>


Cluster 2 turned out to be the same set of coordinates shared among 8 zip codes/areas. It means that the original dataset that I downloaded could be further cleaned. However, I don't want to modify the actual data in this analysis, I will keep it in my future direction: to combine those zip codes/areas in the same coordinate. <br>


Cluster 3 contains the largest number of areas (71 zip codes/areas), including the most two key areas in my town: Urbana (61801) and Champaign (61820). These two zip codes are the 'center' of our town. From the Great Chicago map, we can see that cluster 3 (teal color dots) clearly represent the 'downtown' areas. The top 10 venues of cluster 3 also proved it: you see bars and restaurants, etc. I also noticed that my future address's zip code locates in this cluster 3, so is my current zip code. I am happy to see that the future living environment in Great Chicago is not too different from what I have for now! <br>


Cluster 4 contains 6 areas. The green dots in Chicago map represent this cluster. This area is also a suburb style, mostly containing park, trail, fields, etc. <br>


Cluster 5, the 2nd largest cluster, contains 63 zip codes/areas. The orange dots represent this cluster. Urbana (61821) and Savoy (61874) are in this cluster too. From what I know, these two zip codes are the residential areas. In Chicago area, we can see that these orange dots 'surround' the downtown areas (teal dots, cluster 3). The top 10 venue table also proved it: we see all types of venues associated with essential needs in life, but they are different from cluster 3 where you see mostly restaurants there. In cluster 5 we see more other business types. <br>



### PART6: Conclusion

In this project, I performed an unsupervised machine learning analysis to cluster the areas in Chicago and Champaign-Urbana-Savoy. From the analysis, I was able to successfully use the venue information pulled from Foursquare to cluster the three major types of neighborhooods: Downtown, Residential, and Outdoors. In the search of 'twin' neighborhood, I found that my current zipcode, 61801, is in the 'downtown' cluster (which is true). My future zipcode falls into the same cluster. <br>
To conclude, I have obtained a good understanding of both my current living place and my future living place.<br>